<a href="https://colab.research.google.com/github/GomideB/ExemploDoc/blob/main/OFICIAL_SOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Instalação das bibliotecas necessárias (apenas uma vez)
# --- Seção de Configuração Inicial e Instalação de Dependências ---
# Esta seção é crucial para preparar o ambiente do Google Colab.
# Ela garante que todas as bibliotecas necessárias para o Agente de IA e a interação com as APIs do Google
# sejam instaladas e configuradas corretamente.

# Instalação de bibliotecas:
# O comando '%pip' é específico do ambiente de notebooks (como Google Colab)
# e é equivalente a 'pip install'.
# '-q': Modo silencioso, suprime a maioria das mensagens de saída.
# '--upgrade': Atualiza as bibliotecas para as versões mais recentes.
# '--force-reinstall': Força a reinstalação, útil para garantir que as versões corretas sejam carregadas
# após possíveis conflitos ou instalações anteriores.
# 'google-genai': Biblioteca oficial para interagir com o modelo Gemini (Google AI Studio).
# 'google-adk': Google Agent Development Kit, um framework para construir e gerenciar agentes de IA.
# 'aiohttp': Biblioteca para fazer requisições HTTP assíncronas, utilizada para interagir com a API do Google Maps.
%pip -q install google-genai google-adk aiohttp

In [ ]:
# Reinício do Ambiente de Execução:
# Em muitos casos, após instalar ou atualizar bibliotecas em ambientes como o Colab,
# é necessário reiniciar o kernel para que as novas versões das bibliotecas sejam efetivamente carregadas.
# 'os.kill(os.getpid(), 9)': Este comando encerra o processo Python atual (o kernel do Colab),
# forçando um reinício do ambiente.
# Após isso, o Colab geralmente informa que o ambiente foi reiniciado e você precisa executar
# as próximas células manualmente.
# IMPORTANTE: COLOQUE ESTA CÉLULA SEPARADA DO RESTO DO CÓDIGO.
# Execute-a, aguarde o Colab reiniciar, e SÓ ENTÃO execute a próxima célula com o restante do código.
import os
os.kill(os.getpid(), 9)

In [2]:
# Após o Colab reiniciar, execute o restante do código na próxima célula.

# --- ATENÇÃO: COLOQUE TODO O CÓDIGO ABAIXO EM UMA NOVA CÉLULA NO COLAB ---

# --- Configurações Iniciais ---
import os
from google.colab import userdata

# SUPRESSÃO DE AVISOS DO ASYNCIO (Repetição para garantir após o restart)
import logging
logging.getLogger('asyncio').setLevel(logging.CRITICAL) # <--- Linha para suprimir os avisos

# Configura as API Keys do Google Gemini e Google Maps
# Certifique-se de que 'GOOGLE_API_KEY' e 'Maps_API'
# estão adicionadas nas "Secrets" do Colab (ícone de chave no painel esquerdo)
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
# CORREÇÃO: Usando 'Maps_API' como variável de ambiente para consistência
# e atribuição a 'Maps_API_KEY' para uso nas funções.
# A secret no Colab deve ser salva como 'Maps_API'
os.environ["GOOGLE_MAPS_API"] = userdata.get('GOOGLE_MAPS_API')
Maps_API_KEY = os.environ["GOOGLE_MAPS_API"] # Acessa a chave do env

In [5]:
# Importações
import asyncio
import aiohttp
import google.genai # Importa a biblioteca genai completa
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
# CORREÇÃO: 'Google Search' é uma ferramenta, não 'Google Search' com espaço.
from google.adk.tools import google_search # Importa a ferramenta de busca do Google
from typing import List, Dict, Optional
import nest_asyncio
from datetime import date
import textwrap
from IPython.display import display, Markdown, HTML # HTML para exibir URLs clicáveis
import warnings

warnings.filterwarnings("ignore")
nest_asyncio.apply() # Essencial para rodar asyncio em notebooks do Colab

# Inicializa o cliente Gemini UMA VEZ
client = google.genai.Client()

In [6]:
# --- Funções Auxiliares Comuns ---
def executar_agente(agent: Agent, mensagem: str) -> str:
    """
    Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final.
    Usa um 'for' loop normal, pois 'runner.run()' retorna um gerador síncrono.
    """
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=mensagem)])
    resposta = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text:
                    resposta += part.text + "\n"
    return resposta

def to_markdown(text):
    """Função auxiliar para exibir texto formatado em Markdown no Colab."""
    text = text.replace('•', ' *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [7]:
# --- Definição e Instanciação Única dos Agentes ---
agente_triagem = Agent(
    name="agente_triagem",
    model="gemini-2.0-flash",
    instruction="""Você é um assistente de primeiros socorros.
    Pergunte ao usuário sobre o que ele está sentindo para entender a situação de emergência.
    Mantenha a conversa breve e foque em obter uma descrição clara dos sintomas.
    Após a descrição inicial, **pergunte:** "Você sente mais alguma dor ou algo diferente que eu deva saber?"
    """,
    description="Agente para coletar os sintomas iniciais do usuário."
)

In [8]:
agente_coleta_sintomas_adicionais = Agent(
    name="agente_coleta_sintomas_adicionais",
    model="gemini-2.0-flash",
    instruction="""Você é um assistente de saúde.
    Sua tarefa é consolidar informações de sintomas. Receberá um resumo dos sintomas iniciais
    e uma descrição de sintomas adicionais fornecidos pelo usuário.
    Crie uma descrição única, clara e completa de TODOS os sintomas do paciente,
    sem incluir nenhuma pergunta ou instrução adicional. Se o usuário indicou que não há
    sintomas adicionais, apenas forneça o resumo dos sintomas iniciais.
    """,
    description="Agente para coletar e consolidar sintomas adicionais do usuário."
)

In [10]:
agente_busca_primeiros_socorros = Agent(
    name="agente_busca_primeiros_socorros",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Utilize a ferramenta de busca do Google (Google Search) para encontrar
    informações confiáveis e atualizadas sobre como tratar a condição médica
    descrita pelo usuário. Foque em fontes oficiais de saúde, como
    ministérios da saúde, hospitais renomados e organizações de primeiros
    socorros.  Limite a busca a 5 resultados mais relevantes.""",
    description="Agente para buscar informações de primeiros socorros no Google.",
    # CORREÇÃO: O nome da ferramenta deve ser 'Google Search', não 'Google Search'
    tools=[google_search]
)

In [27]:
agente_planejamento_acoes = Agent(
    name="agente_planejamento_acoes",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise os resultados da busca do Google fornecidos e crie um plano
    conciso e lógico de ações de primeiros socorros a serem recomendadas
    ao usuário. O plano deve ser claro, objetivo e priorizar as ações mais
    urgentes. Inclua, se necessário, orientações sobre quando procurar
    ajuda médica profissional e como proceder até lá. Escreva da forma mais direta e enxuta possível""",
    description="Agente para planejar as ações de primeiros socorros.",
    tools=[google_search]
)

In [28]:
agente_orientacao_detalhada = Agent(
    name="agente_orientacao_detalhada",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base no plano de ações fornecido, gere instruções detalhadas e fáceis
    de seguir para o usuário. Inclua, se possível, links para vídeos do YouTube
    que demonstrem as técnicas de primeiros socorros recomendadas.
    Mencione a importância de procurar ajuda médica profissional e
    oriente o usuário sobre como proceder até lá.""",
    description="Agente para gerar orientações detalhadas de primeiros socorros."
)

In [29]:
agente_avaliacao_emergencia = Agent(
    name="agente_avaliacao_emergencia",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Analise a descrição dos sintomas do usuário e o plano de ações de
    primeiros socorros para determinar a gravidade da situação.
    Se a situação for de emergência (e.g., suspeita de infarto,
    dificuldade respiratória grave, perda de consciência), instrua o usuário
    a ligar imediatamente para o serviço de emergência (192 - SAMU).
    Caso contrário, oriente o usuário a procurar atendimento médico
    em um hospital ou clínica, fornecendo o endereço e a rota.""",
    description="Agente para avaliar a gravidade da situação e recomendar a ação apropriada."
)

In [30]:
agente_localizacao_ajuda_medica = Agent(
    name="agente_localizacao_ajuda_medica",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base na localização fornecida, você pode raciocinar sobre a necessidade de hospitais e médicos.
    No entanto, a busca real e a geração de rotas serão feitas por funções auxiliares, que te entregarão os dados.""",
    description="Agente para raciocinar sobre a necessidade de localizar ajuda médica."
)

In [31]:
agente_rota_navegacao = Agent(
    name="agente_rota_navegacao",
    model="gemini-2.0-flash",
    instruction="""Você é um especialista em primeiros socorros.
    Com base nos hospitais e na sua localização, você pode raciocinar sobre a necessidade de rotas.
    No entanto, a geração de links de rotas será feita por funções auxiliares, que te entregarão os dados.""",
    description="Agente para raciocinar sobre rotas para hospitais."
)

In [32]:
# --- Funções Auxiliares para a API do Google Maps (Permanecem assíncronas) ---
async def buscar_hospitais_proximos(latitude: float, longitude: float, raio: int = 50) -> List[Dict]:
    """
    Busca hospitais próximos a uma dada localização usando a API Google Places Nearby Search.
    Retorna no máximo 4 resultados.
    """
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=hospital&key={Maps_API_KEY}&language=pt"
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url) as response:
                response.raise_for_status()
                data = await response.json()
                if data["status"] == "OK":
                    hospitais = []
                    for result in data["results"]:
                        hospitais.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"),
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return hospitais[:4] # Limita a 4 hospitais
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados hospitais na área especificada.")
                    return []
                else:
                    print(f"Erro ao buscar hospitais: {data['status']}")
                    return []
        except aiohttp.ClientError as e:
            print(f"Erro de requisição HTTP: {e}")
            return []
        except Exception as e:
            print(f"Erro inesperado: {e}")
            return []

async def buscar_medicos_proximos(latitude: float, longitude: float, raio: int = 50) -> List[Dict]:
    """
    Busca médicos/clínicas próximos a uma dada localização usando a API Google Places Nearby Search.
    Retorna no máximo 4 resultados.
    """
    url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={raio * 1000}&type=doctor&key={Maps_API_KEY}&language=pt"
    async with aiohttp.ClientSession() as session:
        try:
            async with session.get(url) as response:
                response.raise_for_status()
                data = await response.json()
                if data["status"] == "OK":
                    medicos = []
                    for result in data["results"]:
                        medicos.append({
                            "nome": result.get("name"),
                            "endereco": result.get("vicinity"),
                            "localizacao": result.get("geometry", {}).get("location"),
                            "rating": result.get("rating")
                        })
                    return medicos[:4] # Limita a 4 médicos
                elif data["status"] == "ZERO_RESULTS":
                    print("Não foram encontrados médicos na área especificada.")
                    return []
                else:
                    print(f"Erro ao buscar médicos: {data['status']}")
                    return []
        except aiohttp.ClientError as e:
            print(f"Erro de requisição HTTP: {e}")
            return []
        except Exception as e:
            print(f"Erro inesperado: {e}")
            return []

async def gerar_rotas_Maps(origem_lat: float, origem_long: float, destinos: List[Dict]) -> List[str]:
    """
    Gera URLs para rotas no Google Maps. Retorna no máximo 4 links.
    """
    rotas = []
    # Usamos o fatiamento aqui para garantir que não tentaremos gerar rotas para mais de 4 destinos
    for destino in destinos[:4]: # Limita os destinos a 4 para gerar rotas
        if "localizacao" in destino and "lat" in destino["localizacao"] and "lng" in destino["localizacao"]:
            destino_lat = destino["localizacao"]["lat"]
            destino_long = destino["localizacao"]["lng"]
            # Ajuste da URL para um formato mais comum e clicável no Google Maps
            url = f"https://www.google.com/maps/dir/?api=1&origin={origem_lat},{origem_long}&destination={destino_lat},{destino_long}&travelmode=driving"
            rotas.append(url)
        else:
            print(f"Localização inválida para destino: {destino}")
    return rotas[:4] # Garante que apenas 4 rotas sejam retornadas, caso a lista de destinos tivesse mais que 4 e a geração falhasse em alguns.

In [ ]:
# --- Agente Orquestrador (Função Python que controla o fluxo) ---
async def orquestrador_primeiros_socorros(mensagem_usuario: str, latitude_usuario: float, longitude_usuario: float):
    """
    Orquestra a interação entre os diferentes agentes de IA para fornecer assistência de primeiros socorros.
    """
    print("⚕️ Iniciando o Sistema de Primeiros Socorros com Agentes ⚕️")

    # 1. Agente de Triagem:
    print("\n--- 🗣️ Chamando Agente de Triagem... ---\n")
    resposta_triagem = executar_agente(agente_triagem, mensagem_usuario)
    display(to_markdown(f"**Agente de Triagem:** {resposta_triagem}"))
    print("--------------------------------------------------------------")

    # NOVA ETAPA: Coleta de sintomas adicionais do usuário.
    mensagem_adicional_usuario = input("Você sente mais alguma dor ou algo diferente que eu deva saber? (Digite 'não' se não houver): ")

    # 2. Agente de Coleta de Sintomas Adicionais:
    print("\n--- 🗣️ Chamando Agente de Coleta de Sintomas Adicionais... ---\n")
    sintomas_consolidados = executar_agente(agente_coleta_sintomas_adicionais,
                                            f"Resumo dos sintomas iniciais:\n{resposta_triagem}\n\nSintomas adicionais informados pelo usuário:\n{mensagem_adicional_usuario}")
    display(to_markdown(f"**Sintomas Consolidados:**\n{sintomas_consolidados}"))
    print("--------------------------------------------------------------")

    # 3. Agente de Busca de Primeiros Socorros:
    print("\n--- 🔍 Chamando Agente de Busca de Primeiros Socorros... ---\n")
    resultados_busca = executar_agente(agente_busca_primeiros_socorros, sintomas_consolidados)
    display(to_markdown(f"**Resultados da Busca:**\n{resultados_busca}"))
    print("--------------------------------------------------------------")

    # 4. Agente de Planejamento de Ações:
    print("\n--- 📝 Chamando Agente de Planejamento de Ações... ---\n")
    plano_acoes = executar_agente(agente_planejamento_acoes, resultados_busca)
    display(to_markdown(f"**Plano de Ações:**\n{plano_acoes}"))
    print("--------------------------------------------------------------")

    # 5. Agente de Geração de Orientação Detalhada:
    print("\n--- 💡 Chamando Agente de Geração de Orientação Detalhada... ---\n")
    orientacao_detalhada = executar_agente(agente_orientacao_detalhada, plano_acoes)
    display(to_markdown(f"**Orientação Detalhada:**\n{orientacao_detalhada}"))
    print("--------------------------------------------------------------")

    # 6. Agente de Localização de Ajuda Médica e Funções de Maps:
    print("\n--- 🏥 Buscando Hospitais e Médicos Próximos... ---\n")
    executar_agente(agente_localizacao_ajuda_medica, "Coletando localização do usuário e buscando locais de ajuda.")

    hospitais = await buscar_hospitais_proximos(latitude_usuario, longitude_usuario)
    medicos = await buscar_medicos_proximos(latitude_usuario, longitude_usuario)

    print("\n**Hospitais Próximos (Top 4):**")
    if hospitais:
        for hospital in hospitais:
            display(to_markdown(f"- **{hospital.get('nome')}** ({hospital.get('endereco')}, Avaliação: {hospital.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados hospitais próximos."))

    print("\n**Médicos Próximos (Top 4):**")
    if medicos:
        for medico in medicos:
            display(to_markdown(f"- **{medico.get('nome')}** ({medico.get('endereco')}, Avaliação: {medico.get('rating', 'N/A')})"))
    else:
        display(to_markdown("Não foram encontrados médicos próximos."))
    print("--------------------------------------------------------------")

    # 7. Agente de Rota e Navegação e Funções de Maps:
    print("\n--- 🗺️ Gerando Rotas para Hospitais... ---\n")
    executar_agente(agente_rota_navegacao, "Gerando links de rotas para os hospitais encontrados.")

    rotas_hospitais = await gerar_rotas_Maps(latitude_usuario, longitude_usuario, hospitais)
    if rotas_hospitais:
        for i, rota in enumerate(rotas_hospitais):
            # Garantir que temos um nome de hospital para a rota
            hospital_nome = hospitais[i].get('nome', 'Hospital Desconhecido') if i < len(hospitais) else 'Destino'
            display(to_markdown(f"Rota para **{hospital_nome}**:"))
            display(HTML(f'<a href="{rota}" target="_blank">{rota}</a>'))
    else:
        display(to_markdown("Não foi possível gerar rotas para os hospitais."))
    print("--------------------------------------------------------------")

    # 8. Agente de Avaliação de Emergência:
    print("\n--- 🚨 Chamando Agente de Avaliação de Emergência... ---\n")
    recomendacao_emergencia = executar_agente(agente_avaliacao_emergencia, f"Descrição dos sintomas: {sintomas_consolidados}\nPlano de ações: {plano_acoes}")
    display(to_markdown(f"**Avaliação de Emergência:**\n{recomendacao_emergencia}"))
    print("--------------------------------------------------------------")



# --- Execução Principal ---
async def main():
    """
    Função principal para iniciar o fluxo do sistema de primeiros socorros.
    """
    mensagem_usuario = input("❓ Por favor, descreva o que você está sentindo (ex: 'Dor no peito e formigamento no braço esquerdo'): ")

    # Para testes, usando localização fixa. Em um projeto real, você obteria a localização do usuário.
    # Exemplo: Campinas, SP
    latitude_usuario = -22.9056
    longitude_usuario = -47.0608

    # Chama o orquestrador principal com a mensagem e localização do usuário.
    await orquestrador_primeiros_socorros(mensagem_usuario, latitude_usuario, longitude_usuario)

if __name__ == "__main__":
    await main()
